# Regression

Regression is a logical extension of classification. Rather than just predicting a single value from a set of values, regression is the act of predicting a real number (or continuous variable) from a set of features (represented as numbers).

Regression can be harder than classification because, from a mathematical perspective, there are an infinite number of possible output values. Furthermore, we aim to optimize some metric of error between the predicted and true value, as opposed to an accuracy rate. Aside from that, regression and classification are fairly similar. For this reason, we will see a lot of the same underlying concepts applied to regression as we did with classification.

## Use Cases

The following is a small set of regression use cases that can get you thinking about potential regression problems in your own domain:

* Predicting movie viewership: Given information about a movie and the movie-going public, such as how many people have watched the trailer or shared it on social media, you might want to predict how many people are likely to watch the movie when it comes out.

* Predicting company revenue: Given a current growth trajectory, the market, and seasonality, you might want to predict how much revenue a company will gain in the future.

* Predicting crop yield: Given information about the particular area in which a crop is grown, as well as the current weather throughout the year, you might want to predict the total crop yield for a particular plot of land.

## Regression Models in MLlib

There are several fundamental regression models in MLlib. This list is current as of Spark 2.4 but will grow:

* [Linear regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression)
* [Generalized linear regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#generalized-linear-regression)
* [Decision tree regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-regression)
* [Random forest regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-regression)
* [Gradient-boosted tree regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-regression)
* [Survival regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#survival-regression)
* [Isotonic regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#isotonic-regression)

Here we will cover:
* A simple explanation of a few of these models and the intuition behind their algorithms
* Model hyperparameters (the different ways that we can initialize the model)
* Training parameters (parameters that affect how the model is trained)
* Prediction parameters (parameters that affect how predictions are made)

You can search over the hyperparameters and training parameters using a `ParamGrid` as we saw before.

## Model Scalability

The regression models in MLlib all scale to large datasets. Table below is a simple model scalability scorecard that will help you in choosing the best model for your particular task (if scalability is your core consideration). These will depend on your configuration, machine size, and other factors.

|Model	|Number features|	Training examples|
|--|--|--|
|Linear regression|1 to 10 million|No limit|
|Generalized linear regression|4,096|No limit|
|Decision trees|1,000s|No limit|
|Random forest|10,000s|No limit|
|Gradient-boosted trees|1,000s|No limit|
|Survival regression|1 to 10 million|No limit|
|Isotonic regression|N/A|Millions|

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import Pretty as disp
hint = 'https://raw.githubusercontent.com/soltaniehha/Big-Data-Analytics-for-Business/master/docs/hints/'  # path to hints on GitHub

Let’s read in some sample data that we will use throughout the notebook:

In [ ]:
# the following line gets the bucket name attached to our cluster
bucket = spark._jsc.hadoopConfiguration().get("fs.gs.system.bucket")

# specifying the path to our bucket where the data is located (no need to edit this path anymore)
data = "gs://" + bucket + "/notebooks/jupyter/data/"
print(data)

In [ ]:
small_df = spark.read.load(data + "regression")

small_df.cache()
print("sales datasets consists of {} rows.".format(small_df.count()))
small_df.show()

## Linear Regression

Linear regression assumes that a linear combination of your input features (the sum of each feature multiplied by a weight) results along with an amount of Gaussian error in the output. This linear assumption (along with Gaussian error) does not always hold true, but it does make for a simple, interpretable model that’s hard to overfit. Like logistic regression, Spark implements `ElasticNet` regularization for this, allowing you to mix *L1* and *L2* regularization.

### Model Hyperparameters

Linear regression has the same model hyperparameters as logistic regression. See below:

Model hyperparameters are configurations that determine the basic structure of the model itself. The following hyperparameters are available for linear regression:

**`elasticNetParam`** (default: 0.0)

The ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. 

**`epsilon`** (default: 1.35)

The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber

**`featuresCol`** (default: features)

Features column name.

**`labelCol`** (default: label)

Label column name. 

**`predictionCol`** (default: prediction)

Prediction column name.


**`loss`** (default: squaredError)

The loss function to be optimized. Supported options: squaredError, huber.

**`maxBlockSizeInMB`** (default: 0.0)

Maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0.


**`fitIntercept`** (default: True)

Whether to fit an intercept term.

**`regParam`** (default: 0.0)

A value ≥ 0. that determines how much weight to give to the regularization term in the objective function. Choosing a value here is again going to be a function of noise and dimensionality in our dataset. In a pipeline, try a wide range of values (e.g., 0, 0.01, 0.1, 1).

**`solver`** (default: auto)

The solver algorithm for optimization. Supported options: auto, normal, l-bfgs.

**`standardization`** (default: True)

Can be true or false, whether or not to standardize the inputs before passing them into the model.

### Training Parameters

Linear regression also shares all of the same training parameters from logistic regression. See below:

Training parameters are used to specify how we perform our training. Here are the training parameters for logistic regression.

**`maxIter`** (default: 100)

Max number of iterations (>= 0).


**`tol`** (default: 1e-06)

This convergence tolerance specifies a threshold by which changes in parameters show that we optimized our weights enough, and can stop iterating. It lets the algorithm stop before `maxIter` iterations. The default value is 1.0E-6. This also shouldn’t be the first parameter you look to tune.

**`weightCol`** (undefined)

The name of a weight column used to weigh certain rows more than others. This can be a useful tool if you have some other measure of how important a particular training example is and have a weight associated with it. For example, you might have 10,000 examples where you know that some labels are more accurate than others. You can weigh the labels you know are correct more than the ones you don’t. If this is not set or empty, we treat all instance weights as 1.0.

### Example

Here’s a short example of using linear regression on our sample dataset:

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
print(lr.explainParams())

In [ ]:
lrModel = lr.fit(small_df)

### Training Summary

Just as in logistic regression, we get detailed training information back from our model. The summary method returns a summary object with several fields. Let’s go through these in turn. 

* The residuals are simply the weights for each of the features that we input into the model. 
* The objective history shows how our training is going at every iteration. 
* The root mean squared error is a measure of how well our line is fitting the data, determined by looking at the distance between each predicted value and the actual value in the data. 
* The R-squared variable is a measure of the proportion of the variance of the predicted variable that is captured by the model.

There are a number of metrics and summary information that may be relevant to your use case. This section demonstrates the API, but does not comprehensively cover every metric (consult the API documentation for more information).

Print the coefficients and intercept for linear regression:

In [ ]:
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

Summarize the model over the training set and print out some metrics:

In [ ]:
trainingSummary = lrModel.summary

In [ ]:
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))

Residuals:

In [ ]:
trainingSummary.residuals.show()

# Your turn

Using a new dataset find a linear regression model that fits the data best.

Dataset: [Online News Popularity Data Set](https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity) from UC Irvine datasets

Attribute Information:

Number of Attributes: 61 (58 predictive attributes, 2 non-predictive, 1 goal field) 

Attribute Information: 

0. url: URL of the article (non-predictive) 
1. timedelta: Days between the article publication and the dataset acquisition (non-predictive) 
2. n_tokens_title: Number of words in the title 
3. n_tokens_content: Number of words in the content 
4. n_unique_tokens: Rate of unique words in the content 
5. n_non_stop_words: Rate of non-stop words in the content 
6. n_non_stop_unique_tokens: Rate of unique non-stop words in the content 
7. num_hrefs: Number of links 
8. num_self_hrefs: Number of links to other articles published by Mashable 
9. num_imgs: Number of images 
10. num_videos: Number of videos 
11. average_token_length: Average length of the words in the content 
12. num_keywords: Number of keywords in the metadata 
13. data_channel_is_lifestyle: Is data channel 'Lifestyle'? 
14. data_channel_is_entertainment: Is data channel 'Entertainment'? 
15. data_channel_is_bus: Is data channel 'Business'? 
16. data_channel_is_socmed: Is data channel 'Social Media'? 
17. data_channel_is_tech: Is data channel 'Tech'? 
18. data_channel_is_world: Is data channel 'World'? 
19. kw_min_min: Worst keyword (min. shares) 
20. kw_max_min: Worst keyword (max. shares) 
21. kw_avg_min: Worst keyword (avg. shares) 
22. kw_min_max: Best keyword (min. shares) 
23. kw_max_max: Best keyword (max. shares) 
24. kw_avg_max: Best keyword (avg. shares) 
25. kw_min_avg: Avg. keyword (min. shares) 
26. kw_max_avg: Avg. keyword (max. shares) 
27. kw_avg_avg: Avg. keyword (avg. shares) 
28. self_reference_min_shares: Min. shares of referenced articles in Mashable 
29. self_reference_max_shares: Max. shares of referenced articles in Mashable 
30. self_reference_avg_sharess: Avg. shares of referenced articles in Mashable 
31. weekday_is_monday: Was the article published on a Monday? 
32. weekday_is_tuesday: Was the article published on a Tuesday? 
33. weekday_is_wednesday: Was the article published on a Wednesday? 
34. weekday_is_thursday: Was the article published on a Thursday? 
35. weekday_is_friday: Was the article published on a Friday? 
36. weekday_is_saturday: Was the article published on a Saturday? 
37. weekday_is_sunday: Was the article published on a Sunday? 
38. is_weekend: Was the article published on the weekend? 
39. LDA_00: Closeness to LDA topic 0 
40. LDA_01: Closeness to LDA topic 1 
41. LDA_02: Closeness to LDA topic 2 
42. LDA_03: Closeness to LDA topic 3 
43. LDA_04: Closeness to LDA topic 4 
44. global_subjectivity: Text subjectivity 
45. global_sentiment_polarity: Text sentiment polarity 
46. global_rate_positive_words: Rate of positive words in the content 
47. global_rate_negative_words: Rate of negative words in the content 
48. rate_positive_words: Rate of positive words among non-neutral tokens 
49. rate_negative_words: Rate of negative words among non-neutral tokens 
50. avg_positive_polarity: Avg. polarity of positive words 
51. min_positive_polarity: Min. polarity of positive words 
52. max_positive_polarity: Max. polarity of positive words 
53. avg_negative_polarity: Avg. polarity of negative words 
54. min_negative_polarity: Min. polarity of negative words 
55. max_negative_polarity: Max. polarity of negative words 
56. title_subjectivity: Title subjectivity 
57. title_sentiment_polarity: Title polarity 
58. abs_title_subjectivity: Absolute subjectivity level 
59. abs_title_sentiment_polarity: Absolute polarity level 
60. shares: Number of shares (target)

The data is in a file named, **OnlineNewsPopularity.csv**

First upload the data to your Google Cloud Storage and then load it to a PySpark DataFrame. Then show the first couple of rows and its schema. You can also cache it for a better performance:

In [ ]:
# Your answer goes here

In [ ]:
# SOLUTION: Uncomment and execute the cell below to get help
#disp(hint + '12-01-load')

Define an RFormula that uses all of the columns as features except the one(s) that are not numerical and call it `supervised`:

In [ ]:
# Your answer goes here

In [ ]:
# SOLUTION: Uncomment and execute the cell below to get help
#disp(hint + '12-01-RFormula')

Fit the RFormula transformer and call it `fittedRF`:

In [ ]:
# Your answer goes here

In [ ]:
# SOLUTION: Uncomment and execute the cell below to get help
#disp(hint + '12-01-fittedRF')

Using `fittedRF` transform our `df` DataFrame. Call this `preparedDF`:

In [ ]:
# Your answer goes here

In [ ]:
# SOLUTION: Uncomment and execute the cell below to get help
#disp(hint + '12-01-preparedDF')

Print the first couple of rows of `preparedDF`:

In [ ]:
preparedDF.select('features', 'label').show(2, False)

Below we will retrieve the name of the columns used to make our feature vector and store them in a pandas DataFrame. Notice that we don't have any binary terms, so we have dropped that from our metadata extraction:

In [ ]:
featureCols = pd.DataFrame(preparedDF.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]).sort_values("idx")

featureCols = featureCols.set_index('idx')
featureCols.head()

Instantiate a linear regression, call it `lr`:

In [ ]:
# Your answer goes here

In [ ]:
# SOLUTION: Uncomment and execute the cell below to get help
#disp(hint + '12-01-lr')

Fit the model using `preparedDF`. Call this model `lrModel`: 

In [ ]:
# Your answer goes here

In [ ]:
# SOLUTION: Uncomment and execute the cell below to get help
#disp(hint + '12-01-lrModel')

* Print coefficients
* Using the model summary output some of its components and assess the goodness of fit

In [ ]:
# Your answer goes here

In [ ]:
# SOLUTION: Uncomment and execute the cell below to get help
#disp(hint + '12-01-summ')

* Try again but this time divid the data into train and test and assess the model on the unseen data
* Design a pipeline and perform hyper-parameter tunning

## Decision Trees

Decision trees as applied to regression work fairly similarly to decision trees applied to classification. The main difference is that decision trees for regression output a single number per leaf node instead of a label (as we saw with classification). The same interpretability properties and model structure still apply. In short, rather than trying to train coeffiecients to model a function, decision tree regression simply creates a tree to predict the numerical outputs. This is of significant consequence because unlike generalized linear regression, we can predict nonlinear functions in the input data. This also creates a significant risk of overfitting the data, so we need to be careful when tuning and evaluating these models.

We also covered decision trees in the previous class. For more information on this topic, consult [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/).

### Model Hyperparameters

The model hyperparameters that apply decision trees for regression are the same as those for classification except for a slight change to the impurity parameter. See notebook from previous class for more information on the other hyperparameters:

**impurity**

The impurity parameter represents the metric (information gain) for whether or not the model should split at a particular leaf node with a particular value or keep it as is. The only metric currently supported for regression trees is “variance.”

### Training Parameters

In addition to hyperparameters, classification and regression trees also share the same training parameters. 

**Example**

Here’s a short example of using a decision tree regressor:

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
print(dtr.explainParams())
dtrModel = dtr.fit(small_df)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = small_df.randomSplit([0.8, 0.2])

# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="features")

# Train model.  This also runs the indexer.
model = dt.fit(trainingData)

Predicting and evaluating on test set:

In [ ]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(metricName="rmse")

rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

# Summary
print(model)

# Your turn

Repeat this with the Online News Popularity Data Set:

In [ ]:
# Your answer goes here

Make predictions on test set and check the model performance against it:

In [ ]:
# Your answer goes here

In [ ]:
# Your answer goes here

* What kind of improvements can you make to better this model?
